# Evaluate automatic coding

In [2]:
# get each coding

import glob
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score, accuracy_score

limit = 350 # WHY LIMIT???
clean_na_dim23 = False

# Load annotations
df_gpt = pd.read_excel('../data/sample_for_check_GPT3.xlsx').fillna('NA')[:limit]
df_elin = pd.read_excel('../data/human_annotation/ELINsample_for_check_human.xlsx').fillna('NA')[:limit]
df_lena = pd.read_excel('../data/human_annotation/Lenasample_for_check_human.xlsx').fillna('NA')[:limit]
df_oscar = pd.read_excel('../data/human_annotation/Oscarsample_for_check_human.xlsx').fillna('NA')[:limit]
masks = dict()

# Output a .csv / .xlxs table for each dim, cleaning dim2 and dim3 from NA
for dim in 'dim1 dim2 dim3'.split():

    names = ['elin', 'lena', 'oscar']

    data = {'elin': df_elin[dim], 'lena': df_lena[dim], 'oscar': df_oscar[dim]}
    out = pd.DataFrame(data=data)
    
    # Keep only dim2 and dim3 examples where all the annotators provided a label
    # dim1 is excluded because NA is a possible label in that category
    if dim in {'dim2', 'dim3'}:
        if clean_na_dim23:
            masks[dim] = (out.elin != 'NA') & (out.lena!="NA") & (out.oscar!="NA") 
        else:
            masks[dim] = out.elin.notnull() # all values are True
        out2 = out[masks[dim]]
    else:
        out2 = out
    
    # Write table to file
    #out2.to_excel('../data/human_annotation/{}.xlsx'.format(dim), index=None)
    #out2.to_csv('../data/human_annotation/{}'.format(dim), index=None, header=None)

print(df_elin.shape)
print(df_oscar.shape)
print(df_lena.shape)
print(df_gpt.shape)
print(df_elin.tail(1).iloc[0,0])
print(df_oscar.tail(1).iloc[0,0])
print(df_lena.tail(1).iloc[0,0])
print(df_gpt.tail(1).iloc[0,0])
for dim in 'dim2 dim3'.split():
    print(masks[dim].sum())

(350, 5)
(350, 5)
(350, 5)
(350, 5)
333163
333163
333163
333163
350
350


/Users/llupo/miniconda3/envs/mentalenv/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/llupo/miniconda3/envs/mentalenv/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/llupo/miniconda3/envs/mentalenv/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [11]:
# ! wc -l ./data/human_annotation/${i} ./data/human_annotation/*csv

wc: ./data/human_annotation/: read: Is a directory
     350 ./data/human_annotation/dim1.prediction.csv
     350 ./data/human_annotation/dim2.prediction.csv
     350 ./data/human_annotation/dim3.prediction.csv
    1050 total


In [132]:
# ! /Users/dirkhovy/Dropbox/tools/MACE/MACE --prefix ../data/GPT/human_annotation/dim1 ../data/GPT/human_annotation/dim1.csv
# ! /Users/dirkhovy/Dropbox/tools/MACE/MACE --prefix ../data/GPT/human_annotation/dim2 ../data/GPT/human_annotation/dim2.csv
# ! /Users/dirkhovy/Dropbox/tools/MACE/MACE --prefix ../data/GPT/human_annotation/dim3 ../data/GPT/human_annotation/dim3.csv

Reading CSV file '../data/GPT/human_annotation/dim1.csv'
....................100
....................200
....................300
.........
stats:
	350 instances,
	6 labels [PASSIVE, NA, ACTIVE_POS_OTHER, ACTIVE_NEG, ACTIVE_POS_CHALLENGING, ACTIVE_POS_CARING],
	3 annotators

Running Variational Bayes EM training with the following settings:
	50 iterations
	10 restarts
	smoothing = 0.0016666666666666668
	alpha = 0.5
	beta = 0.5

Restart 1
initial log marginal likelihood = -1555.6882106001829
final log marginal likelihood = -1351.0009273885416

Restart 2
initial log marginal likelihood = -1553.1490471943941
final log marginal likelihood = -1351.0009277253812

Restart 3
initial log marginal likelihood = -1572.0079877178498
final log marginal likelihood = -1351.0009276184364

Restart 4
initial log marginal likelihood = -1542.5678174738118
final log marginal likelihood = -1351.0009269094794

Restart 5
initial log marginal likelihood = -1522.559650468198
final log marginal likelihood = -1351.

In [3]:
def adapt_to_dim1_mode(dim1_mode, labels):
        if dim1_mode == 'binary':
            labels = [x[:3] for x in labels]
        elif dim1_mode == 'reduced':
            labels = [x[:10] for x in labels]
        return labels

dim1_mode = '' #binary/reduced

for dim in 'dim1 dim2 dim3'.split():
    print(dim)

    # read GPT labels
    gpt_label = df_gpt[dim].values
    # adapt to reduced and binary dim1
    if dim == 'dim1':
        gpt_label = adapt_to_dim1_mode(dim1_mode, gpt_label)
    # account for excluded items
    else:    
        gpt_label = df_gpt[dim][masks[dim]].values

    # read aggregated human labels
    with open('../data/human_annotation/{}.prediction'.format(dim)) as dimagg:
        dim_gpt_agg = [x.strip() for x in dimagg.readlines()]
    # adapt to reduced and binary dim1
    if dim == 'dim1':
        dim_gpt_agg = adapt_to_dim1_mode(dim1_mode, dim_gpt_agg)
  
    # create tables to store agreement outcomes
    conf_kappa = pd.DataFrame(columns=names+['aggregate', 'GPT'], index=names+['aggregate', 'GPT']).fillna(1.0)
    conf_raw = pd.DataFrame(columns=names+['aggregate', 'GPT'], index=names+['aggregate', 'GPT']).fillna(1.0)

    # read human' labels and compare
    human = pd.read_excel('../data/human_annotation/{}.xlsx'.format(dim)).fillna('NA')
    for i, name1 in enumerate(names):
        label1 = human[name1].values
        # adapt to reduced and binary dim1
        if dim == 'dim1':
            label1 = adapt_to_dim1_mode(dim1_mode, label1)
        
        # compare name's label with GPT
        ck = cohen_kappa_score(label1, gpt_label)
        agg = accuracy_score(label1, gpt_label)
        
        for j, name2 in enumerate(names[i+1:]):
            label2 = human[name2].values
            # adapt to reduced and binary dim1
            if dim == 'dim1':
                label2 = adapt_to_dim1_mode(dim1_mode, label2)
                    
            if name1 != name2:
                ck1 = cohen_kappa_score(label1, label2)
                agg1 = accuracy_score(label1, label2)
                conf_kappa[name1][name2] = ck1
                conf_raw[name1][name2] = agg1
                conf_kappa[name2][name1] = ck1
                conf_raw[name2][name1] = agg1
                
        conf_kappa['GPT'][name1] = ck
        conf_raw['GPT'][name1] = agg
        conf_kappa[name1]['GPT'] = ck
        conf_raw[name1]['GPT'] = agg
        ck_label_agg = cohen_kappa_score(dim_gpt_agg, label1)
        agg_label_agg = accuracy_score(dim_gpt_agg, label1)
        
        # print('{}:aggregate, kappa={:.2f}, agreement={:.2f}'.format(name1, ck_label_agg, agg_label_agg))
        
        conf_kappa[name1]['aggregate'] = ck_label_agg
        conf_raw[name1]['aggregate'] = agg_label_agg
        conf_kappa['aggregate'][name1] = ck_label_agg
        conf_raw['aggregate'][name1] = agg_label_agg

    ck_agg = cohen_kappa_score(dim_gpt_agg, gpt_label)
    agg_agg = accuracy_score(dim_gpt_agg, gpt_label)
    
    # ßprint('aggregate:GPT, kappa={:.2f}, agreement={:.2f}'.format(ck_agg, agg_agg))
    
    conf_kappa['GPT']['aggregate'] = ck_agg
    conf_raw['GPT']['aggregate'] = agg_agg
    conf_kappa['aggregate']['GPT'] = ck_agg
    conf_raw['aggregate']['GPT'] = agg_agg
    
    conf_kappa['mean_human'] = conf_kappa[names].mean(axis=1)
    conf_raw['mean_human'] = conf_raw[names].mean(axis=1)
    for name in names:
        conf_kappa.mean_human[name] = (conf_kappa[names].loc[name].sum() - 1.0) / 2
        conf_raw.mean_human[name] = (conf_raw[names].loc[name].sum() - 1.0) / 2
        
    print('RAW:')
    print(conf_raw)
    print('KAPPA:')
    print(conf_kappa)
    print()

dim1
elin:aggregate, kappa=0.67, agreement=0.75
lena:aggregate, kappa=0.75, agreement=0.82
oscar:aggregate, kappa=0.78, agreement=0.84
aggregate:GPT, kappa=0.41, agreement=0.55
RAW:
               elin      lena     oscar  aggregate       GPT  mean_human
elin       1.000000  0.608571  0.617143   0.748571  0.494286    0.612857
lena       0.608571  1.000000  0.668571   0.817143  0.545714    0.638571
oscar      0.617143  0.668571  1.000000   0.840000  0.528571    0.642857
aggregate  0.748571  0.817143  0.840000   1.000000  0.548571    0.801905
GPT        0.494286  0.545714  0.528571   0.548571  1.000000    0.522857
KAPPA:
               elin      lena     oscar  aggregate       GPT  mean_human
elin       1.000000  0.486754  0.499290   0.672779  0.354546    0.493022
lena       0.486754  1.000000  0.533719   0.750879  0.388609    0.510237
oscar      0.499290  0.533719  1.000000   0.781927  0.364316    0.516505
aggregate  0.672779  0.750879  0.781927   1.000000  0.406302    0.735195
GPT     